# TSFresh exploration

In [20]:
from tsfresh import select_features, extract_relevant_features, extract_features
from tsfresh.utilities.dataframe_functions import impute
from get_processed_data import get_processed_data, get_train_test_split
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from tsfresh.transformers import RelevantFeatureAugmenter
from sklearn.metrics import classification_report


In [34]:
train_test_split  = get_train_test_split(
  'malte', 60 * 15, 60 * 3
)

3

In [15]:

ts_train = []
y_train = pd.Series() # 0 = not hard, 1 = hard
i = 0

ts_test = []
y_test = pd.Series() # 0 = not hard, 1 = hard


for split_i, split in enumerate(train_test_split):
  for difficulty, windows in split.items():
    for window in windows:
      data  = pd.DataFrame(window)
      data = data.drop("PacketCounter", axis=1)
      data["id"] = i
      if split_i == 0:
        ts_train.append(data)
        y_train[i] = difficulty == "hard"
      else:
        ts_test.append(data)
        y_test[i] = difficulty == "hard"
      i += 1

df_ts_train = pd.concat(ts_train)
df_ts_test = pd.concat(ts_test)


In [18]:
X_test = pd.DataFrame(index=y_test.index)
X_train = pd.DataFrame(index=y_train.index)

In [19]:
ppl = Pipeline([
            ('augmenter', RelevantFeatureAugmenter(column_id='id', column_sort='SampleTimeFine', column_kind=None, column_value=None)),
            ('classifier', RandomForestClassifier()),
            ])
ppl.set_params(augmenter__timeseries_container=df_ts_train)
ppl.fit(X_train, y_train)

Feature Extraction: 100%|██████████| 20/20 [14:23<00:00, 43.18s/it]
/Users/malte/Developer/data-science-for-wearables/.venv/lib/python3.12/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['Quat_Z__query_similarity_count__query_None__threshold_0.0'
 'FreeAcc_X__query_similarity_count__query_None__threshold_0.0'
 'FreeAcc_Y__query_similarity_count__query_None__threshold_0.0'
 'FreeAcc_Z__query_similarity_count__query_None__threshold_0.0'
 'Quat_W__query_similarity_count__query_None__threshold_0.0'
 'Quat_X__query_similarity_count__query_None__threshold_0.0'
 'Quat_Y__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


Pipeline(steps=[('augmenter',
                 RelevantFeatureAugmenter(column_id='id',
                                          column_sort='SampleTimeFine',
                                          timeseries_container=     SampleTimeFine    Quat_W    Quat_X    Quat_Y    Quat_Z  FreeAcc_X  \
0                 0  0.810841  0.198744  0.136467 -0.533313  -0.018069   
1             16667  0.811080  0.198380  0.135855 -0.533242   0.136214   
2             33334  0.811371  0.197772  0.135500 -0.533115   0.340523   
3             50001  0.811737  0.196823  0.135474 -0.532915   0.4563...
899        98985313  0.404666  0.595681 -0.487348  0.493872   0.134930   

     FreeAcc_Y  FreeAcc_Z   id  
0    -0.214065   0.148417    0  
1    -0.373040  -0.007256    0  
2    -0.459667  -0.135375    0  
3    -0.452044  -0.139472    0  
4    -0.495046   0.041724    0  
..         ...        ...  ...  
895  -0.121798   0.030168  111  
896  -0.147419   0.019240  111  
897  -0.059146   0.150910  111  
898   0.026887   0.280197  111  
899  -0.052465   0.250441  111  

[100800 rows x 9 columns])),
                ('classifier', RandomForestClassifier())])

In [21]:
ppl.set_params(augmenter__timeseries_container=df_ts_test)
y_pred = ppl.predict(X_test)
print(classification_report(y_test, y_pred))

Feature Extraction: 100%|██████████| 20/20 [00:42<00:00,  2.14s/it]

              precision    recall  f1-score   support

       False       0.75      1.00      0.86         3
        True       1.00      0.91      0.95        11

    accuracy                           0.93        14
   macro avg       0.88      0.95      0.90        14
weighted avg       0.95      0.93      0.93        14

